Import all packages in a cell

In [1]:
# Import all packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings




In [2]:
# read the dataset and set skipinitialspace to true to be able to .replace
df = pd.read_csv('./HouseholderAtRisk(1).csv', skipinitialspace=True)

# show all columns 
print(df.info())

# Task 1 question 1
# Show proportion of high risk
risks = df['AtRisk'].value_counts()
print("Proportion of high risk = " + str(risks[0]/len(df)))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39999 entries, 0 to 39998
Data columns (total 18 columns):
ID                        39999 non-null int64
Age                       39032 non-null float64
WorkClass                 39027 non-null object
Weighting                 38707 non-null float64
Education                 39027 non-null object
NumYearsEducation         39027 non-null float64
Marital-Status            39027 non-null object
Occupation                39013 non-null object
Relationship              39027 non-null object
Race                      45 non-null object
Gender                    39027 non-null object
CapitalLoss               39027 non-null float64
CapitalGain               39027 non-null float64
CapitalAvg                39027 non-null float64
NumWorkingHoursPerWeek    39027 non-null float64
Sex                       39027 non-null float64
CountryOfOrigin           39969 non-null object
AtRisk                    39999 non-null object
dtypes: float64(8), int

In [ ]:
# Replace inconsistency in CountryOfOrigin
df['CountryOfOrigin'] = df['CountryOfOrigin'].replace("US", "USA").replace("United-States", "USA")

# Removing Gender to use numerical binary for Sex where 0 = Male, 1 = Female
df.drop('Gender', axis=1, inplace=True)

# Replace missing value in Age with mean value
meanAge = df['Age'].mean()
df['Age'].fillna(meanAge, inplace=True)
df.loc[df['Age']==-1, 'Age'] = meanAge

# Round off age
df['Age'] = df['Age'].astype(int)

# Drop race as there are 39954 NaN vs 45 labelled classes
df.drop('Race', axis=1, inplace=True)

# Replace missing value in NumYearsEducation wih mean value
df['NumYearsEducation'].fillna(df['NumYearsEducation'].mean(), inplace=True)

# Replace missing value in Marital Status with mode value
df['Marital-Status'].fillna(df['Marital-Status'].mode()[0], inplace=True)

# Replace missing value in Sex with mode value
df['Sex'].fillna(df['Sex'].mode()[0], inplace=True)

# Sets upper boundary of 90 hours work week in NumWorkingHoursPerWeek and fills with mean value
meanWorkHours = df['NumWorkingHoursPerWeek'].mean()
df['NumWorkingHoursPerWeek'] = df['NumWorkingHoursPerWeek'].where(df['NumWorkingHoursPerWeek'] <= 90, meanWorkHours)  
# Replace missing value in NumWorkingHoursPerWeek with mean value
df['NumWorkingHoursPerWeek'].fillna(meanWorkHours, inplace=True)
# Round off hours                             
df['NumWorkingHoursPerWeek'] = df['NumWorkingHoursPerWeek'].astype(int)
                                    



In [57]:
#print(df.isnull().any()) # Has NaN in all that returned true

#print(np.where(df['Relationship'].isna() == True))
#print(np.where(df['Sex'].isna() == True))
x = (np.where(df['Relationship'].isna() == True))
y = (np.where(df['Sex'].isna() == True))
z = (np.where(df['NumYearsEducation'].isna() == True))

# Relation, Sex and NumYearsEducation has the same rows where value is NaN
e = [(a == b) for a, b in zip(x,y)]
f = [(c == d) for c, d in zip(x,z)]
print(np.any(f))

#print(df['Sex'].isna().sum())
#print(df['NumYearsEducation'].isna().sum())
#print(df['Age'].isna().sum())





True
